<a href="https://colab.research.google.com/github/MinsooKwak/RAG/blob/main/test/chunking/chunking_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- langchain 제공
  - CharacterTextSplitter() : 문자 기반
    - Tiktoken : 토큰 단위
  - RecursiveCharacterTextSplitter() : 재귀 + 문자 기반
  - TokenTextSplitter() : 토큰 수 기준 분할
- Semantic chunk

- 청킹시 고려사항
  1. 문맥 손실
  2. 청크 경계 처리 : 청크 나누는 과정에서 문장 잘리지 않도록 주의
  - 해결 방안 : 청크 경계가 자연스러운 언어 단위(문장, 문단)에 맞춰지도록 조정하고, 필요한 경우 청크간 중복 일부 허용

In [ ]:
# Arxiv 문서 로더 > 설치  # pymupdf : arxiv.org 사이트에서 다운로드한 PDF 파일을 텍스트 형식)
#!pip install -qU langchain-community arxiv pymupdf

In [ ]:
# PDF 가져올 때
from langchain.document_loaders import PyPDFLoader

"""
# PDF 가져오기
loaders = [
    PyPDFLoader(""),
    PyPDFLoader(""),
]

docs2 = []
for loader in loaders:
  docs2.extend(loader.load())
"""

In [2]:
from langchain_community.document_loaders import ArxivLoader

# Query 에 검색하고자 하는 논문의 주제를 입력
loader = ArxivLoader(
    query="Agent",
    load_max_docs= 10,              # 최대 문서 수
    load_all_available_meta=True,   # 메타데이터 전체 로드 여부
)

In [5]:
# 문서 로드 결과출력
docs = loader.load()
docs

[Document(metadata={'Published': '2024-07-11', 'Title': 'EvoAgent: Towards Automatic Multi-Agent Generation via Evolutionary Algorithms', 'Authors': 'Siyu Yuan, Kaitao Song, Jiangjie Chen, Xu Tan, Dongsheng Li, Deqing Yang', 'Summary': 'The rise of powerful large language models (LLMs) has spurred a new trend in\nbuilding LLM-based autonomous agents for solving complex tasks, especially\nmulti-agent systems. Despite the remarkable progress, we notice that existing\nworks are heavily dependent on human-designed frameworks, which greatly limits\nthe functional scope and scalability of agent systems. How to automatically\nextend the specialized agent to multi-agent systems to improve task-solving\ncapability still remains a significant challenge. In this paper, we introduce\nEvoAgent, a generic method to automatically extend expert agents to multi-agent\nsystems via the evolutionary algorithm, thereby improving the effectiveness of\nLLM-based agents in solving tasks. Specifically, we cons

In [6]:
print(len(docs))

10


In [32]:
print(docs[0].page_content)

EVOAGENT: Towards Automatic Multi-Agent
Generation via Evolutionary Algorithms
Siyu Yuan1∗, Kaitao Song2∗†,
Jiangjie Chen1, Xu Tan2, Dongsheng Li2, Deqing Yang1†
Fudan University1, Microsoft Research Asia2
syyuan21@m.fudan.edu.cn, {kaitaosong, xuta, dongsli}@microsoft.com
{jjchen19,yangdeqing}@fudan.edu.cn
https://evo-agent.github.io
Abstract
The rise of powerful large language models (LLMs) has spurred a new trend in
building LLM-based autonomous agents for solving complex tasks, especially
multi-agent systems. Despite the remarkable progress, we notice that existing
works are heavily dependent on human-designed frameworks, which greatly limits
the functional scope and scalability of agent systems. How to automatically extend
the specialized agent to multi-agent systems to improve task-solving capability
still remains a significant challenge. In this paper, we introduce EVOAGENT, a
generic method to automatically extend expert agents to multi-agent systems via
the evolutionary algorit

In [7]:
# 첫번째 문서 메타데이터 출력
docs[0].metadata

{'Published': '2024-07-11',
 'Title': 'EvoAgent: Towards Automatic Multi-Agent Generation via Evolutionary Algorithms',
 'Authors': 'Siyu Yuan, Kaitao Song, Jiangjie Chen, Xu Tan, Dongsheng Li, Deqing Yang',
 'Summary': 'The rise of powerful large language models (LLMs) has spurred a new trend in\nbuilding LLM-based autonomous agents for solving complex tasks, especially\nmulti-agent systems. Despite the remarkable progress, we notice that existing\nworks are heavily dependent on human-designed frameworks, which greatly limits\nthe functional scope and scalability of agent systems. How to automatically\nextend the specialized agent to multi-agent systems to improve task-solving\ncapability still remains a significant challenge. In this paper, we introduce\nEvoAgent, a generic method to automatically extend expert agents to multi-agent\nsystems via the evolutionary algorithm, thereby improving the effectiveness of\nLLM-based agents in solving tasks. Specifically, we consider the existin

In [20]:
docs[0].type

'Document'

In [28]:
docs[0]

Document(metadata={'Published': '2024-07-11', 'Title': 'EvoAgent: Towards Automatic Multi-Agent Generation via Evolutionary Algorithms', 'Authors': 'Siyu Yuan, Kaitao Song, Jiangjie Chen, Xu Tan, Dongsheng Li, Deqing Yang', 'Summary': 'The rise of powerful large language models (LLMs) has spurred a new trend in\nbuilding LLM-based autonomous agents for solving complex tasks, especially\nmulti-agent systems. Despite the remarkable progress, we notice that existing\nworks are heavily dependent on human-designed frameworks, which greatly limits\nthe functional scope and scalability of agent systems. How to automatically\nextend the specialized agent to multi-agent systems to improve task-solving\ncapability still remains a significant challenge. In this paper, we introduce\nEvoAgent, a generic method to automatically extend expert agents to multi-agent\nsystems via the evolutionary algorithm, thereby improving the effectiveness of\nLLM-based agents in solving tasks. Specifically, we consi

In [9]:
import pandas as pd
import numpy as np

In [30]:
# Document 객체의 내부 데이터를 딕셔너리로 변환하는 함수
def document_to_dict(doc):
    return {
        'metadata': doc.metadata,
        'page_content': doc.page_content,
        'type': doc.type
    }

# 모든 Document 객체를 딕셔너리로 변환
docs_dict = [document_to_dict(doc) for doc in docs]

In [31]:
df2 = pd.json_normalize(
    docs_dict,
    sep='_',
    meta=[
        'page_content', 'type',
        ['metadata', 'Published'],
        ['metadata', 'Title'],
        ['metadata', 'Authors'],
        ['metadata', 'Summary'],
        ['metadata', 'entry_id'],
        ['metadata', 'published_first_time'],
        ['metadata', 'comment'],
        ['metadata', 'journal_ref'],
        ['metadata', 'doi'],
        ['metadata', 'primary_category']
    ],
    record_path=['metadata', 'categories'],  # 'categories'가 리스트이므로 이를 따로 분리
    errors='ignore'
)
df2

,0,page_content,type,metadata_Published,metadata_Title,metadata_Authors,metadata_Summary,metadata_entry_id,metadata_published_first_time,metadata_comment,metadata_journal_ref,metadata_doi,metadata_primary_category
0,cs.AI,EVOAGENT: Towards Automatic Multi-Agent\nGener...,Document,2024-07-11,EvoAgent: Towards Automatic Multi-Agent Genera...,"Siyu Yuan, Kaitao Song, Jiangjie Chen, Xu Tan,...",The rise of powerful large language models (LL...,http://arxiv.org/abs/2406.14228v2,2024-06-20,Work in process,None,None,cs.AI
1,cs.LG,Cooperative Heterogeneous Deep Reinforcement\n...,Document,2020-11-02,Cooperative Heterogeneous Deep Reinforcement L...,"Han Zheng, Pengfei Wei, Jing Jiang, Guodong Lo...",Numerous deep reinforcement learning agents ha...,http://arxiv.org/abs/2011.00791v1,2020-11-02,None,None,None,cs.LG
2,cs.AI,Cooperative Heterogeneous Deep Reinforcement\n...,Document,2020-11-02,Cooperative Heterogeneous Deep Reinforcement L...,"Han Zheng, Pengfei Wei, Jing Jiang, Guodong Lo...",Numerous deep reinforcement learning agents ha...,http://arxiv.org/abs/2011.00791v1,2020-11-02,None,None,None,cs.LG
3,cs.HC,Improving of Robotic Virtual Agent’s errors th...,Document,2023-08-21,Improving of Robotic Virtual Agent's errors th...,"Takahiro Tsumura, Seiji Yamada",One way to improve the relationship between hu...,http://arxiv.org/abs/2304.00247v2,2023-04-01,"13 pages, 4 figures, 5 tables, submitted ICSR2...",None,None,cs.HC
4,math.OC,arXiv:1405.1480v1 [math.OC] 7 May 2014\nOn N...,Document,2014-05-07,On Networks with Active and Passive Agents,Tansel Yucelen,We introduce an active-passive networked multi...,http://arxiv.org/abs/1405.1480v1,2014-05-07,None,None,None,math.OC
5,math.DS,arXiv:1405.1480v1 [math.OC] 7 May 2014\nOn N...,Document,2014-05-07,On Networks with Active and Passive Agents,Tansel Yucelen,We introduce an active-passive networked multi...,http://arxiv.org/abs/1405.1480v1,2014-05-07,None,None,None,math.OC
6,cs.LG,Cooperative and Competitive Biases\nfor Multi-...,Document,2021-01-18,Cooperative and Competitive Biases for Multi-A...,"Heechang Ryu, Hayong Shin, Jinkyoo Park",Training a multi-agent reinforcement learning ...,http://arxiv.org/abs/2101.06890v1,2021-01-18,Accepted as a full paper at the Twentieth Inte...,None,None,cs.LG
7,cs.AI,Cooperative and Competitive Biases\nfor Multi-...,Document,2021-01-18,Cooperative and Competitive Biases for Multi-A...,"Heechang Ryu, Hayong Shin, Jinkyoo Park",Training a multi-agent reinforcement learning ...,http://arxiv.org/abs/2101.06890v1,2021-01-18,Accepted as a full paper at the Twentieth Inte...,None,None,cs.LG
8,cs.MA,Cooperative and Competitive Biases\nfor Multi-...,Document,2021-01-18,Cooperative and Competitive Biases for Multi-A...,"Heechang Ryu, Hayong Shin, Jinkyoo Park",Training a multi-agent reinforcement learning ...,http://arxiv.org/abs/2101.06890v1,2021-01-18,Accepted as a full paper at the Twentieth Inte...,None,None,cs.LG
9,cs.AI,"JOURNAL OF LATEX CLASS FILES, VOL. 18, NO. 9, ...",Document,2023-10-18,Fact-based Agent modeling for Multi-Agent Rein...,"Baofu Fang, Caiming Zheng, Hao Wang","In multi-agent systems, agents need to interac...",http://arxiv.org/abs/2310.12290v1,2023-10-18,None,None,None,cs.AI


In [ ]:
#!pip install langchain --upgrade

In [ ]:
#!pip install pypdf

In [118]:
# PDF 가져올 때
from langchain.document_loaders import PyPDFLoader

# PDF 가져오기
loaders = [
    PyPDFLoader("/content/2306.05685v4.pdf"),
    PyPDFLoader("/content/2406.14562v1.pdf"),
    PyPDFLoader("/content/2406.14563v1.pdf"),
]

docs2 = []
for loader in loaders:
  docs2.extend(loader.load())

In [121]:
len(docs2)

58

In [106]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter

text = df2['page_content'][0]

In [113]:
chunk_size = 500
chunk_overlap = 10

# 문서를 재귀적으로 분할 (세분화된 split 가능)
### ex. \n\n 기준으로 나누고, \n로 분할하고, 마지막으로 character로 분리
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n","\n", " ", ""]
)

# 단순히 주어진 텍스트 문자 단위 분할
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

# TokenTextSplitter
t_splitter = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

# Tiktoken
tik_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    model_name="gpt-3.5-turbo"
)

In [114]:
tik_doc = tik_splitter.split_documents(docs)

# 첫번째 문서의 분할된 청크 크기와 문서 내용 출력
print(len(tik_doc[0].page_content))
tik_doc[0]

1865


Document(page_content='EVOAGENT: Towards Automatic Multi-Agent\nGeneration via Evolutionary Algorithms\nSiyu Yuan1∗, Kaitao Song2∗†,\nJiangjie Chen1, Xu Tan2, Dongsheng Li2, Deqing Yang1†\nFudan University1, Microsoft Research Asia2\nsyyuan21@m.fudan.edu.cn, {kaitaosong, xuta, dongsli}@microsoft.com\n{jjchen19,yangdeqing}@fudan.edu.cn\nhttps://evo-agent.github.io\nAbstract\nThe rise of powerful large language models (LLMs) has spurred a new trend in\nbuilding LLM-based autonomous agents for solving complex tasks, especially\nmulti-agent systems. Despite the remarkable progress, we notice that existing\nworks are heavily dependent on human-designed frameworks, which greatly limits\nthe functional scope and scalability of agent systems. How to automatically extend\nthe specialized agent to multi-agent systems to improve task-solving capability\nstill remains a significant challenge. In this paper, we introduce EVOAGENT, a\ngeneric method to automatically extend expert agents to multi-age

In [115]:
# 두번째 문서의 분할된 청크 크기와 문서 내용 출력
print(len(tik_doc[1].page_content))
tik_doc[1]

568


Document(page_content='Based on the foundation of LLMs,\nmany research works [5, 6, 7, 8, 9, 10, 11] have discovered that by empowering multiple advanced\nskills (e.g., planning, tool, memory and so on), we can develop more powerful autonomous agents to\nsolve more challenging tasks. Therefore, how to design and leverage LLM-based autonomous agents\nto tackle more diverse and complex real-world applications has attracted enormous interest. Generally, many real-world scenarios are usually complex, encompassing a variety of challenging\ntasks that are beyond the capability of a single agent.')

In [122]:
tik_doc2 = tik_splitter.split_documents(docs2)

# 첫번째 문서의 분할된 청크 크기와 문서 내용 출력
print(len(tik_doc2[0].page_content))
tik_doc2[0]

3101


Document(metadata={'source': '/content/2306.05685v4.pdf', 'page': 0}, page_content='Judging LLM-as-a-Judge\nwith MT-Bench and Chatbot Arena\nLianmin Zheng1∗Wei-Lin Chiang1∗Ying Sheng4∗Siyuan Zhuang1\nZhanghao Wu1Yonghao Zhuang3Zi Lin2Zhuohan Li1Dacheng Li13\nEric P. Xing35Hao Zhang12Joseph E. Gonzalez1Ion Stoica1\n1UC Berkeley2UC San Diego3Carnegie Mellon University4Stanford5MBZUAI\nAbstract\nEvaluating large language model (LLM) based chat assistants is challenging due to\ntheir broad capabilities and the inadequacy of existing benchmarks in measuring\nhuman preferences. To address this, we explore using strong LLMs as judges to\nevaluate these models on more open-ended questions. We examine the usage and\nlimitations of LLM-as-a-judge, including position, verbosity, and self-enhancement\nbiases, as well as limited reasoning ability, and propose solutions to mitigate some\nof them. We then verify the agreement between LLM judges and human preferences\nby introducing two benchmarks: MT

In [123]:
# 두번째 문서의 분할된 청크 크기와 문서 내용 출력
print(len(tik_doc2[1].page_content))
tik_doc2[1]

5162


Document(metadata={'source': '/content/2306.05685v4.pdf', 'page': 1}, page_content="Question:If the FED buys bonds in the secondary market(A) the money supply will increase. (B) the money supply will decrease. (C) the money supply will not be affected. (D) the discount rate would be affected. Output the answer only.Assistant B:(A) the money supply will increase.Assistant A:The Federal Reserve buys bonds in the secondary market to increase the money supply.Follow-up Question: How does it affect my daily life? Give 3 examples.Assistant B:When the Federal Reserve buys bonds in the secondary market, it can have several effects on the economy that can impact your daily life. Here are three examples:1. Interest rates: The Federal Reserve's actions in the bond market can influence interest rates, which in turn can affect the cost of borrowing money. For example, if the Federal Reserve buys bonds and drives up demand for them, interest rates on things like mortgages, car loans, and credit card

In [98]:
r_splitter.split_text(text)[0]

'EVOAGENT: Towards Automatic Multi-Agent\nGeneration via Evolutionary Algorithms\nSiyu Yuan1∗, Kaitao Song2∗†,\nJiangjie Chen1, Xu Tan2, Dongsheng Li2, Deqing Yang1†\nFudan University1, Microsoft Research Asia2\nsyyuan21@m.fudan.edu.cn, {kaitaosong, xuta, dongsli}@microsoft.com\n{jjchen19,yangdeqing}@fudan.edu.cn\nhttps://evo-agent.github.io\nAbstract\nThe rise of powerful large language models (LLMs) has spurred a new trend in\nbuilding LLM-based autonomous agents for solving complex tasks, especially'

In [99]:
r_splitter.split_text(text)[1]

'multi-agent systems. Despite the remarkable progress, we notice that existing\nworks are heavily dependent on human-designed frameworks, which greatly limits\nthe functional scope and scalability of agent systems. How to automatically extend\nthe specialized agent to multi-agent systems to improve task-solving capability\nstill remains a significant challenge. In this paper, we introduce EVOAGENT, a\ngeneric method to automatically extend expert agents to multi-agent systems via'

In [100]:
r_splitter.split_text(text)[2]

'the evolutionary algorithm, thereby improving the effectiveness of LLM-based\nagents in solving tasks. Specifically, we consider the existing agent frameworks\nas the initial individual and then apply a series of evolutionary operators (e.g.,\nmutation, crossover, selection, etc.) to generate multiple agents with diverse agent\nsettings. EVOAGENT can be generalized to any LLM-based agent framework,\nand can automatically extend the existing agent framework to multi-agent systems'

In [104]:
len(c_splitter.split_text(text))

1

In [105]:
c_splitter.split_text(text)

['EVOAGENT: Towards Automatic Multi-Agent\nGeneration via Evolutionary Algorithms\nSiyu Yuan1∗, Kaitao Song2∗†,\nJiangjie Chen1, Xu Tan2, Dongsheng Li2, Deqing Yang1†\nFudan University1, Microsoft Research Asia2\nsyyuan21@m.fudan.edu.cn, {kaitaosong, xuta, dongsli}@microsoft.com\n{jjchen19,yangdeqing}@fudan.edu.cn\nhttps://evo-agent.github.io\nAbstract\nThe rise of powerful large language models (LLMs) has spurred a new trend in\nbuilding LLM-based autonomous agents for solving complex tasks, especially\nmulti-agent systems. Despite the remarkable progress, we notice that existing\nworks are heavily dependent on human-designed frameworks, which greatly limits\nthe functional scope and scalability of agent systems. How to automatically extend\nthe specialized agent to multi-agent systems to improve task-solving capability\nstill remains a significant challenge. In this paper, we introduce EVOAGENT, a\ngeneric method to automatically extend expert agents to multi-agent systems via\nthe e

In [108]:
print(len(t_splitter.split_text(text)))

47


In [109]:
t_splitter.split_text(text)[0]

'EVOAGENT: Towards Automatic Multi-Agent\nGeneration via Evolutionary Algorithms\nSiyu Yuan1∗, Kaitao Song2∗†,\nJiangjie Chen1, Xu Tan2, Dongsheng Li2, Deqing Yang1†\nFudan University1, Microsoft Research Asia2\nsyyuan21@m.fudan.edu.cn, {kaitaosong, xuta, dongsli}@microsoft.com\n{jjchen19,yangdeqing}@fudan.edu.cn\nhttps://evo-agent.github.io\nAbstract\nThe rise of powerful large language models (LLMs) has spurred a new trend in\nbuilding LLM-based autonomous agents for solving complex tasks, especially\nmulti-agent systems. Despite the remarkable progress, we notice that existing\nworks are heavily dependent on human-designed frameworks, which greatly limits\nthe functional scope and scalability of agent systems. How to automatically extend\nthe specialized agent to multi-agent systems to improve task-solving capability\nstill remains a significant challenge. In this paper, we introduce EVOAGENT, a\ngeneric method to automatically extend expert agents to multi-agent systems via\nthe ev

In [110]:
t_splitter.split_text(text)[1]

' memory and so on), we can develop more powerful autonomous agents to\nsolve more challenging tasks. Therefore, how to design and leverage LLM-based autonomous agents\nto tackle more diverse and complex real-world applications has attracted enormous interest.\nGenerally, many real-world scenarios are usually complex, encompassing a variety of challenging\ntasks that are beyond the capability of a single agent. To address this point, we notice that human\nsociety is composed of vast individuals, each possessing their unique characteristics. By selecting,\norchestrating, and cooperating with different individuals, humans can form an efficient team group\nto handle complicated missions in the real world. Therefore, there has been an increasing trend\nto develop multi-agent collaboration frameworks (e.g., MetaGPT [10], AutoGen [12], Camel [13],\n∗The first two authors have equal contributions. This work was done when the first author was an intern at\nMicrosoft Research Asia.\n† Correspon

In [ ]:
#!pip install -qU langchain_experimental langchain_openai

In [62]:
import os
from config import OPEN_AI_API_KEY
os.environ["OPENAI_API_KEY"] = OPEN_AI_API_KEY

In [83]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# OpenAI 임베딩을 사용하여 의미론적 청크 분할기를 초기화합니다.
text_splitter = SemanticChunker(OpenAIEmbeddings(),
                                # percentile 기준
                                breakpoint_threshold_type="percentile",
                                breakpoint_threshold_amount=70,
                                # 표준편차 기준
                                #breakpoint_threshold_type="standard_deviation",
                                #breakpoint_threshold_amount=1.25,
                                )

In [84]:
#문서 단위로 분할
chunks = text_splitter.split_text(text)

In [85]:
print(chunks[0])

EVOAGENT: Towards Automatic Multi-Agent
Generation via Evolutionary Algorithms
Siyu Yuan1∗, Kaitao Song2∗†,
Jiangjie Chen1, Xu Tan2, Dongsheng Li2, Deqing Yang1†
Fudan University1, Microsoft Research Asia2
syyuan21@m.fudan.edu.cn, {kaitaosong, xuta, dongsli}@microsoft.com
{jjchen19,yangdeqing}@fudan.edu.cn
https://evo-agent.github.io
Abstract
The rise of powerful large language models (LLMs) has spurred a new trend in
building LLM-based autonomous agents for solving complex tasks, especially
multi-agent systems. Despite the remarkable progress, we notice that existing
works are heavily dependent on human-designed frameworks, which greatly limits
the functional scope and scalability of agent systems. How to automatically extend
the specialized agent to multi-agent systems to improve task-solving capability
still remains a significant challenge. In this paper, we introduce EVOAGENT, a
generic method to automatically extend expert agents to multi-agent systems via
the evolutionary algorit

In [86]:
print(chunks[1])

Based on the foundation of LLMs,
many research works [5, 6, 7, 8, 9, 10, 11] have discovered that by empowering multiple advanced
skills (e.g., planning, tool, memory and so on), we can develop more powerful autonomous agents to
solve more challenging tasks. Therefore, how to design and leverage LLM-based autonomous agents
to tackle more diverse and complex real-world applications has attracted enormous interest. Generally, many real-world scenarios are usually complex, encompassing a variety of challenging
tasks that are beyond the capability of a single agent.


In [87]:
print(chunks[2])

To address this point, we notice that human
society is composed of vast individuals, each possessing their unique characteristics. By selecting,
orchestrating, and cooperating with different individuals, humans can form an efficient team group
to handle complicated missions in the real world. Therefore, there has been an increasing trend
to develop multi-agent collaboration frameworks (e.g., MetaGPT [10], AutoGen [12], Camel [13],
∗The first two authors have equal contributions. This work was done when the first author was an intern at
Microsoft Research Asia.


In [88]:
# text_splitter를 사용하여 분할
docs = text_splitter.create_documents([text])
print(docs[0].page_content)  # 분할된 문서 중 첫 번째 문서의 내용을 출력

EVOAGENT: Towards Automatic Multi-Agent
Generation via Evolutionary Algorithms
Siyu Yuan1∗, Kaitao Song2∗†,
Jiangjie Chen1, Xu Tan2, Dongsheng Li2, Deqing Yang1†
Fudan University1, Microsoft Research Asia2
syyuan21@m.fudan.edu.cn, {kaitaosong, xuta, dongsli}@microsoft.com
{jjchen19,yangdeqing}@fudan.edu.cn
https://evo-agent.github.io
Abstract
The rise of powerful large language models (LLMs) has spurred a new trend in
building LLM-based autonomous agents for solving complex tasks, especially
multi-agent systems. Despite the remarkable progress, we notice that existing
works are heavily dependent on human-designed frameworks, which greatly limits
the functional scope and scalability of agent systems. How to automatically extend
the specialized agent to multi-agent systems to improve task-solving capability
still remains a significant challenge. In this paper, we introduce EVOAGENT, a
generic method to automatically extend expert agents to multi-agent systems via
the evolutionary algorit